<a href="https://colab.research.google.com/github/Masum06/Text2App/blob/master/text2app_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Model_Testing/nl_sar_train.csv', error_bad_lines=False)
valid = pd.read_csv('/content/drive/MyDrive/Model_Testing/nl_sar_valid.csv', error_bad_lines=False)
test = pd.read_csv('/content/drive/MyDrive/Model_Testing/nl_sar_test.csv', error_bad_lines=False)
# df = df.drop(['NL'], axis=1)

In [ ]:
data = {
    "train": train,
    "test": test,
    "valid": valid
}

In [ ]:
!mkdir -p data/

In [ ]:
!rm /data/*.txt

rm: cannot remove '/data/*.txt': No such file or directory


In [ ]:
for key in data.keys():
  fp_input = open("data/src_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['NL'].values)))
  fp_input.close

  fp_input = open("data/tgt_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['SAR'].values)))
  fp_input.close

In [ ]:
!unzip /content/drive/"My Drive"/OpenNMT-py-legacy.zip

Archive:  /content/drive/My Drive/OpenNMT-py-legacy.zip
585499a4509b1c2dac7683b7588f3f544f89119e
   creating: OpenNMT-py-legacy/
  inflating: OpenNMT-py-legacy/.gitignore  
  inflating: OpenNMT-py-legacy/.travis.yml  
  inflating: OpenNMT-py-legacy/CHANGELOG.md  
  inflating: OpenNMT-py-legacy/CONTRIBUTING.md  
  inflating: OpenNMT-py-legacy/LICENSE.md  
  inflating: OpenNMT-py-legacy/README.md  
   creating: OpenNMT-py-legacy/available_models/
  inflating: OpenNMT-py-legacy/available_models/example.conf.json  
   creating: OpenNMT-py-legacy/config/
  inflating: OpenNMT-py-legacy/config/config-rnn-summarization.yml  
  inflating: OpenNMT-py-legacy/config/config-transformer-base-1GPU.yml  
  inflating: OpenNMT-py-legacy/config/config-transformer-base-4GPU.yml  
   creating: OpenNMT-py-legacy/data/
  inflating: OpenNMT-py-legacy/data/README.md  
  inflating: OpenNMT-py-legacy/data/ggnnsrc.txt  
  inflating: OpenNMT-py-legacy/data/ggnnsrcvocab.txt  
  inflating: OpenNMT-py-legacy/data/ggn

In [ ]:
!pip install OpenNMT-py

     |████████████████████████████████| 215kB 5.7MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 748.8MB 19kB/s 
     |████████████████████████████████| 2.6MB 35.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 276kB 32.8MB/s 
     |████████████████████████████████| 1.2MB 33.1MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=75036becb0688ed2e0f1038576e5640ee251904eae2f08c445fd4f64a44c7cbc
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=0f2a887054e5a5ab2c6c971f22f5b79d0dbef8b89cc71cfb818950663f340b84
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735

In [ ]:
!export CUDA_VISIBLE_DEVICES

### Train with Copy Mechanism

In [ ]:
!mkdir -p /content/drive/"My Drive"/text2app_models

In [ ]:
!python OpenNMT-py-legacy/preprocess.py -train_src data/src_train.txt -train_tgt data/tgt_train.txt \
-valid_src data/src_valid.txt -valid_tgt data/tgt_valid.txt -src_seq_length 85 -tgt_seq_length 50 -save_data data/preprocessed_data

# -dynamic_dict 

[2021-03-24 02:32:17,752 INFO] Extracting features...
[2021-03-24 02:32:17,754 INFO]  * number of source features: 0.
[2021-03-24 02:32:17,754 INFO]  * number of target features: 0.
[2021-03-24 02:32:17,754 INFO] Building `Fields` object...
[2021-03-24 02:32:17,755 INFO] Building & saving training data...
[2021-03-24 02:32:17,876 INFO] Building shard 0.
[2021-03-24 02:32:18,396 INFO]  * saving 0th train data shard to data/preprocessed_data.train.0.pt.
[2021-03-24 02:32:19,078 INFO]  * tgt vocab size: 114.
[2021-03-24 02:32:19,078 INFO]  * src vocab size: 126.
[2021-03-24 02:32:19,080 INFO] Building & saving validation data...
[2021-03-24 02:32:19,103 INFO] Building shard 0.
[2021-03-24 02:32:19,138 INFO]  * saving 0th valid data shard to data/preprocessed_data.valid.0.pt.


In [ ]:
!python OpenNMT-py-legacy/train.py \
-encoder_type brnn \
-global_attention mlp \
-world_size 1 -gpu_ranks 0 \
-train_steps 30000 -save_checkpoint_steps 500 \
-valid_steps 500 \
-data data/preprocessed_data \
-train_from /content/drive/MyDrive/Model_Testing/text2app_models/model_step_15000.pt \
-save_model /content/drive/MyDrive/Model_Testing/text2app_models/model

# -copy_attn \
# -reuse_copy_attn \
# -copy_loss_by_seqlength \

[2021-03-24 02:42:23,220 INFO] Loading checkpoint from /content/drive/MyDrive/Model_Testing/text2app_models/model_step_15000.pt
[2021-03-24 02:42:23,277 INFO] Loading vocab from checkpoint at /content/drive/MyDrive/Model_Testing/text2app_models/model_step_15000.pt.
[2021-03-24 02:42:23,278 INFO]  * src vocab size = 126
[2021-03-24 02:42:23,278 INFO]  * tgt vocab size = 114
[2021-03-24 02:42:23,278 INFO] Building model...
[2021-03-24 02:42:26,056 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(126, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(114, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rn

#Test

In [ ]:
best_model_dir = "/content/drive/MyDrive/Model_Testing/text2app_models/model_step_30000.pt"

In [ ]:
!python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
data/src_test.txt -output data/pred.txt -replace_unk -verbose -beam_size 1

Streaming output truncated to the last 5000 lines.
SENT 1001: ['create', 'an', 'app', 'consisting', 'of', 'a', 'password', 'box', ',', 'an', 'audio', 'with', 'source', 'string0', ',', 'a', 'video', 'with', 'source', 'string1', ',', 'and', 'a', 'switch', '.', 'when', 'the', 'switch', 'is', 'clicked,', 'play', 'the', 'video', '.']
PRED 1001: <start_comp> <passwordtextbox> <player> string0 </player> <video_player> string1 </video_player> <switch> switch1 </switch> <end_comp> <code> <switch1flipped> <video_player1> <start> </video_player1> </switch1flipped> </code>
PRED SCORE: 0.0000

[2021-03-24 03:04:52,988 INFO] 
SENT 1002: ['make', 'an', 'app', 'containing', 'a', 'label', 'with', 'text', 'string0', ',', 'an', 'audio', 'with', 'source', 'string1', ',', 'a', 'switch', ',', 'and', 'a', 'label', 'with', 'text', 'string2', '.', 'if', 'the', 'switch', 'is', 'clicked,', 'set', 'label', 'text', 'to', 'string3', '.']
PRED 1002: <start_comp> <label> string0 </label> <player> string1 </player> <s

In [ ]:
f = open("data/pred.txt")
lines = f.readlines()
f.close()

In [ ]:
pred = pd.DataFrame({'pred':lines})

In [ ]:
len(pred)

2000

In [ ]:
len(test)

2000

In [ ]:
pred

,pred
0,<start_comp> <timepicker> <end_comp>\n
1,<start_comp> <passwordtextbox> <timepicker> <e...
2,<start_comp> <passwordtextbox> <switch> string...
3,<start_comp> <video_player> string0 </video_pl...
4,<start_comp> <player> string0 </player> <playe...
...,...
1995,<start_comp> <textbox> <switch> switch1 </swit...
1996,<start_comp> <button> string0 </button> <label...
1997,<start_comp> <textbox> <text2speech> <password...
1998,<start_comp> <passwordtextbox> <passwordtextbo...


In [ ]:
test[['SAR']]

,SAR
0,<start_comp> <timepicker> <end_comp>
1,<start_comp> <passwordtextbox> <timepicker> <e...
2,<start_comp> <passwordtextbox> <switch> string...
3,<start_comp> <video_player> string0 </video_pl...
4,<start_comp> <player> string0 </player> <playe...
...,...
1995,<start_comp> <textbox> <switch> switch1 </swit...
1996,<start_comp> <button> string0 </button> <label...
1997,<start_comp> <textbox> <text2speech> <password...
1998,<start_comp> <passwordtextbox> <passwordtextbo...


In [ ]:
def trim(s):
  return ' '.join(s.split())

In [ ]:
import numpy as np

In [ ]:
test_np = test['SAR'].apply(trim).to_numpy()
pred_np = pred['pred'].apply(trim).to_numpy()

In [ ]:
correct = 1*(test_np == pred_np)

In [ ]:
correct.sum()/len(correct)

0.8305

## Single data Test

In [ ]:
def is_Number(test):
  is_number = True
  try:
    float(test)
  except ValueError:
    is_number = False
  return is_number

In [ ]:
import nltk
def get_singular(word):
  p = nltk.PorterStemmer()
  a = p.stem(word)
  return a 

In [ ]:
def NL_mod(nl):
  vis_portion = nl.split(".")[0].split() 
  for i in range(len(vis_portion)):
    if is_Number(vis_portion[i]):
      if float(vis_portion[i]) > 1:
        component = get_singular(vis_portion[i+1])
        mod = ""
        for j in range(int(vis_portion[i])):
          mod += component
          mod += " "
        mod = mod.rstrip()
        rep = vis_portion[i] + " " + vis_portion[i + 1]
        nl = nl.replace(rep, mod, 1)
  return nl.lower()

In [ ]:
best_model_dir = '/content/drive/MyDrive/Model_Testing/text2app_models/model_step_3000.pt'

In [ ]:
def test(nl):
  nl = NL_mod(nl)
  with open("test.txt", "w") as f:
    f.write(nl)
  
  !python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1
  out = open("sar_to_compile.txt", 'r')
  print(out)
  # return out

In [ ]:
nl = """Create an app with noise noise a text to speech , and button STRING0 . When the button is clicked, speak the text in the text box ."""
test(nl)

[2021-03-14 07:49:22,231 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
[2021-03-14 07:49:22,285 INFO] 
SENT 1: ['create', 'an', 'app', 'with', 'noise', 'noise', 'a', 'text', 'to', 'speech', ',', 'and', 'button', 'STRING0', '.', 'when', 'the', 'button', 'is', 'clicked,', 'speak', 'the', 'text', 'in', 'the', 'text', 'box', '.']
PRED 1: <start_vis> <text2speech> <button> STRING0 </button> <end_vis> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>
PRED SCORE: 0.0000

[2021-03-14 07:49:22,285 INFO] PRED AVG SCORE: 0.0000, PRED PPL: 1.0000
<_io.TextIOWrapper name='sar_to_compile.txt' mode='r' encoding='UTF-8'>


In [ ]:
from google.colab import files
files.download('sar_to_compile.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
best_model_dir = '/content/drive/MyDrive/Model_Testing/text2app_models/model_step_30000.pt'

In [ ]:
!python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
single_test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1

[2021-03-24 10:35:00,904 INFO] Translating shard 0.
[2021-03-24 10:35:00,963 INFO] 
SENT 1: ['create', 'an', 'app', 'with', 'a', 'button', ',', 'a', 'textbox', ',', 'and', 'a', 'text2speech', '.', 'when', 'the', 'button', 'is', 'pressed', ',', 'speak', 'the', 'textbox', 'text', '.']
PRED 1: <start_comp> <button> button1 </button> <textbox> <text2speech> <end_comp> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>
PRED SCORE: 0.0000

[2021-03-24 10:35:00,963 INFO] PRED AVG SCORE: 0.0000, PRED PPL: 1.0000


In [ ]:
import subprocess

def tranlsate(NL):
  nl_file = open("single_test.txt", "w")
  nl_file.write(NL)
  nl_file.close()
  s = "python OpenNMT-py-legacy/translate.py -model {} -src single_test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1".format(best_model_dir)
  subprocess.call(s.split())
  f = open("sar_to_compile.txt", 'r')
  sar = f.read().strip()
  f.close()
  subprocess.call('rm single_test.txt'.split())
  subprocess.call('rm sar_to_compile.txt'.split())
  return sar

In [ ]:
NL = "create an app with a button , a textbox , and a text2speech . when the button is pressed , speak the textbox text ."
tranlsate(NL)

'<start_comp> <button> button1 </button> <textbox> <text2speech> <end_comp> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>'

In [ ]:
!wget https://www.dropbox.com/s/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
!unzip OpenNMT-py-legacy.zip

--2021-03-24 10:48:57--  https://www.dropbox.com/s/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/du1w2kkbxkwv788/OpenNMT-py-legacy.zip [following]
--2021-03-24 10:48:57--  https://www.dropbox.com/s/raw/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0e79fdb16f65e24fb4e89df7f5.dl.dropboxusercontent.com/cd/0/inline/BLQTO3Kqwucpta1Pqm4p0QRZiCn3jWfJwqkgQ7btUXdii7_tXgX_Fq-w7ASBEvLgOFN_0RF6wuh4tzhBflvgUpN25XiuDTS0ESKQLjPq0eYY0XJtts_TIgCM4ATZttqM4nRhtpGIdIr6esHFHD2AYRlI/file# [following]
--2021-03-24 10:48:58--  https://uc0e79fdb16f65e24fb4e89df7f5.dl.dropboxusercontent.com/cd/0/inline/BLQTO3Kqwucpta1Pqm4p0QRZiCn3jWfJwqkgQ7btUXdii7_tXgX_Fq-w7ASB